In [ ]:
import warnings ; warnings.filterwarnings('ignore')


import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)

In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-t_ys1jbf/gym-walk_f522bf83f2724a578f9fce8b25fbb973
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-t_ys1jbf/gym-walk_f522bf83f2724a578f9fce8b25fbb973
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=71bbb1c83150ead2e29003a1388db3073f084ef4098e2dc96d18fb36a26bc3d1
  Stored in directory: /tmp/pip-ephem-wheel-cache-p6bck9l3/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
env = gym.make('FrozenLake-v1')
P = env.env.P
init_state = env.reset()
#goal_state = 6
#LEFT, RIGHT = range(2)

In [ ]:
P

Exponentially decaying schedule


In [ ]:
import numpy as np

def decay_schedule(init_value, min_value,decay_ratio, max_steps,log_start=-2, log_base=10):
    decay_steps = int(max_steps * decay_ratio)
    rem_steps = max_steps - decay_steps
    values = np.logspace(log_start, 0, decay_steps,base=log_base, endpoint=True)[::-1]
    values = (values - values.min()) / (values.max() - values.min())
    values = (init_value - min_value) * values + min_value
    values = np.pad(values, (0, rem_steps), 'edge')
    return values




Exploratory Policy Trajectories

In [ ]:
from itertools import count
def generate_trajectory(select_action,Q,epsilon,env,max_steps=200):
    done, trajectory = False, []
    while not done:
        state = env.reset()
        for t in count():
            action = select_action(state,Q,epsilon)
            next_state, reward, done, _ = env.step(action)
            experience = (state, action, reward,
                          next_state, done)
            trajectory.append(experience)
            if done:
                break
            if t >= max_steps - 1:
                trajectory = []
                break
            state = next_state
    return np.array(trajectory,object)



In [ ]:
from tqdm import tqdm

Monte Carlo control

In [ ]:
def mc_control(
    env, gamma=1.0,
    init_alpha=0.5, min_alpha=0.01, alpha_decay_ratio=0.5,
    init_epsilon=1.0, min_epsilon=0.1, epsilon_decay_ratio=0.9,
    n_episodes=3000, max_steps=200, first_visit=True
):
    nS, nA = env.observation_space.n, env.action_space.n
    discounts = np.logspace(0, max_steps, num=max_steps, base=gamma, endpoint=False)


    alphas = decay_schedule(init_alpha, min_alpha, alpha_decay_ratio, n_episodes)
    epsilons = decay_schedule(init_epsilon, min_epsilon, epsilon_decay_ratio, n_episodes)

    Q = np.zeros((nS, nA))
    Q_track = np.zeros((n_episodes, nS, nA))
    pi_track = []

    for e in tqdm(range(n_episodes), leave=False):
        trajectory = generate_trajectory(select_action, Q, epsilons[e], env, max_steps)

        visited = np.zeros((nS, nA), dtype=np.bool_)

        for t, (state, action, reward, _, _) in enumerate(trajectory):
            if visited[state][action] and first_visit:
                continue
            visited[state][action] = True

            n_steps = len(trajectory[t:])
            rewards = np.array([x[2] for x in trajectory[t:]])
            G = np.sum(discounts[:n_steps] * rewards)

            Q[state][action] = Q[state][action] + alphas[e] * (G - Q[state][action])

        Q_track[e] = Q
        pi_track.append(np.argmax(Q, axis=1))

    V = np.max(Q, axis=1)
    pi = {s: a for s, a in enumerate(np.argmax(Q, axis=1))}

    return Q, V, pi, Q_track, pi_track




In [ ]:
import numpy as np

def select_action(state, Q, epsilon):

    nA = Q.shape[1]

    if np.random.rand() < epsilon:

        return np.random.randint(nA)
    else:

        return np.argmax(Q[state])

In [ ]:
optimal_Q, optimal_V, optimal_pi, Q_track, pi_track = mc_control(env)
print("NAME: POPURI SRAVANI")
print("REQNO:212223240117")
print_state_value_function(optimal_Q, P, n_cols=4, prec=2, title='Action-value function:')
print_state_value_function(optimal_V, P, n_cols=4, prec=2, title='State-value function:')
print_policy(optimal_pi, P)


NAME: POPURI SRAVANI
REQNO:212223240117
Action-value function:
| 00 [0.09 0.07 0.06 0.07] | 01 [0.02 0.05 0.02 0.02] | 02 [0.04 0.01 0.09 0.01] | 03 [0.03 0.01 0.   0.  ] |
| 04 [0.1  0.07 0.08 0.04] |           | 06 [0.05 0.04 0.12 0.  ] |           |
| 08 [0.03 0.11 0.08 0.1 ] | 09 [0.19 0.17 0.24 0.09] | 10 [0.13 0.35 0.14 0.04] |           |
|           | 13 [0.14 0.19 0.43 0.2 ] | 14 [0.3  0.46 0.69 0.39] |           |
State-value function:
| 00   0.09 | 01   0.05 | 02   0.09 | 03   0.03 |
| 04    0.1 |           | 06   0.12 |           |
| 08   0.11 | 09   0.24 | 10   0.35 |           |
|           | 13   0.43 | 14   0.69 |           |
Policy:
| 00      < | 01      v | 02      > | 03      < |
| 04      < |           | 06      > |           |
| 08      v | 09      > | 10      v |           |
|           | 13      > | 14      > |           |
